In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
import pandas as pd
import torch
from tqdm import tqdm

In [ ]:
import json

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
test_df = spark.read.parquet("data/df_test.parquet").toPandas()

In [ ]:
display(test_df)

In [ ]:
with open("models/finetuned_scibert_scivocab_uncased_8cats/id2label.json", "r") as f:
    id2label = {int(k): v for k, v in json.load(f).items()}

id2label

In [ ]:
candidate_labels = [id2label[i] for i in range(len(id2label))]
candidate_labels

In [ ]:
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)

In [ ]:
zero_shot_preds = []
batch_size = 1  # Adjust based on memory
texts = test_df["text"].tolist()

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    res_batch = zero_shot(batch, candidate_labels)
    
    if isinstance(res_batch, dict):  # Happens if batch size = 1
        zero_shot_preds.append(res_batch["labels"][0])
    else:
        zero_shot_preds.extend([res["labels"][0] for res in res_batch])